# 시계열 데이터 다루기(기본)

## 1. Data Types and Tools of  Date & Time

In [1]:
# 날짜와 시간을 위한 표준라이브러리 불러오기
from datetime import datetime
now = datetime.now()
print(now)
print(now.year, now.month, now.day)

2019-11-02 18:35:48.316359
2019 11 2


In [ ]:
# 시간의 차이를 표현해보자.
delta = datetime(2019, 11, 2) - datetime(2019, 6, 24, 8, 15)
print(delta)
print(delta.days)
print(delta.seconds)

In [ ]:
# 시간 연산
from datetime import timedelta
start = datetime(2019, 1, 7)
print(start + timedelta(12)) # 12일 더하기
print(start - 2 * timedelta(12)) # 2*12일 빼기

<center><b>Data types of date and time</b></center>

| 자료형 |  설명   |  
|---|:---|
| `date` | 그레고리안 달력을 사용해서 연, 월, 일 저장  |  
| `time` | 하루의 시간을 시, 분, 초, 마이크로초 단위로 저장 |  
| `datetime` | 날짜와 시간을 저장 |
| `timedelta` | 두 datetime 간의 차이(일, 초 마이크로초)를 표현 |  
| `tzinfo` | 지역시간대를 저장 |

### 1.1 문자열 <-> datetime으로 변환하기

In [ ]:
# datatime을 문자열로 변환하기
## datetime 객체와 pandas의 timestamp 객체는 str메서드나 strftime 메서드에 포맷규칙을 넘겨서  문자열로 나타낼 수 있다.
stamp = datetime(2019, 11, 2)
print(str(stamp))
print(stamp.strftime('%Y-%m-%d'))

In [ ]:
# 문자열을 datetime으로 변환하기
strings = '2019-11-02 12:00:00'
from dateutil.parser import parse
print(parse(strings), type(parse(strings)))
import pandas as pd
print(pd.to_datetime(strings), type(pd.to_datetime(strings)))

## 2. 시계열 기초

### 2.1 indexing, selecting, slicing

In [ ]:
# datetime 인덱스로 pandas series 생성
from datetime import datetime
dates = [datetime(2019, 1, 2), datetime(2019, 1, 5),
         datetime(2019, 1, 7), datetime(2019, 1, 8),
         datetime(2019, 1, 10), datetime(2019, 1, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
ts

In [ ]:
ts.index, ts.values

In [ ]:
# slicing
ts[0], ts['2019-01-02']

In [ ]:
ts[1::2]

In [ ]:
ts['2019-01-05':'2019-01-08']

In [ ]:
ts[10<=ts.index.day]

In [ ]:
ts.truncate(after='2019-01-08'), ts.truncate(before='01/08/2019') 
# 날짜형식 : yyyy-mm-dd, yyyy/mm/dd, dd/mm/yyyy, yyyymmdd 모두 가능

In [ ]:
ts['20190110']

### 2.2 중복된 색인을 갖는 시계열

In [ ]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

In [ ]:
# 색인의 중복여부 점검
dup_ts.index.is_unique

In [ ]:
grouped = dup_ts.groupby(dup_ts.index)
grouped.count(), grouped.mean()

### 2.3 날짜 범위, 빈도, 이동

#### 2.3.1 날짜 범위 생성하기

In [ ]:
index = pd.date_range('2019-09-01', '2019-10-31')
#index = pd.date_range(start='2019-09-01',  period=60)
#index = pd.date_range(end='2019-10-31', period=60)

#### 2.3.2 빈도

In [ ]:
from pandas.tseries.offsets import Hour, Minute
pd.date_range('2019-10-30', '2019-11-02', freq='D')  # 빈도 : 매일
# 매월 마지막일 : M, W : 매주, 매시: H, 매분: T 또는 min, 매초 : S, 매밀리초 : L 또는 ms, ...

In [ ]:
# 빈도 앞에 숫자를 두면 해당 빈도의 곱을 생성한다.
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h') 

#### 2.3.3 샘플링

In [ ]:
twodays = pd.Series(np.random.randn(365), index=pd.date_range('2018-01-01', '2019-12-31', freq='2d'))
twodays.head()

In [ ]:
# 다운샘플링
years.resample(rule='A').mean()   # 매년,   
# 매분기 : Q, 그 외에 위의 빈도에서 사용된 부호 이용 가능

In [ ]:
# 업샘플링
onedays = twodays.resample('D').asfreq()  # 빈칸에 NaN이 들어간다.
onedays.head()

In [ ]:
# 보간 : NaN 채우기
onedays_bogan =  twodays.resample('D').ffill()  # 빈칸에 바로 앞의 값이 들어간다.
onedays_bogan.head()

#### 2.3.4 날짜 이동

In [ ]:
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('1/1/2000', periods=4, freq='D'))
ts, ts.shift(2)    # 값이 뒤로 2칸(day) 이동

In [ ]:
ts.shift(-2)   # 값이 앞으로 2칸(day) 이동

In [ ]:
ts / ts.shift(1) - 1    # 값  연산도 가능

In [ ]:
# 빈도수를 주면, 그것을 기준으로 계산
ts, ts.shift(2, freq='90min')    # 3시간(2 x 90 min) 이동

#### 2.3.5 다른 시간대(time zone) 간의 연산

In [ ]:
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')  # B : business day
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

In [ ]:
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')
result = ts1 + ts2
result.index   # 서로 다른 시간대를 연산하면 결과는 utc가 된다.

### 2.4 Moving window

In [ ]:
# 데이터 로드
from sklearn.datasets import load_boston
pd_boston = pd.DataFrame( load_boston().data, columns=load_boston().feature_names,
               index=pd.date_range(start='1/1/2000', end='09/06/2009', freq='W'))
pd_boston.head()

In [ ]:
pd_boston['RM'].plot(label='origin')
pd_boston['RM'].rolling(50).mean().plot(label='50')   # 50 : 샘플 개수
pd_boston['RM'].rolling('30d').mean().plot(label='30d')  # 30일 크기의 이동평균창
plt.legend()